> Topics that will be covered:  

- Data collection:
  - From simple to somewhat challenging
- Data wrangling:
  - Cleaning data
  - Filtering data
  - Combine data
- Data analysis:
  - Data Visualization
  - Descriptive Statistics
  - Univariate & Bivariate analysis

---

> **pandas** method & attributes:  

- `.shape`
- `.loc`
- `.iloc`
- `.info()`
- `.duplicated()`
- `.isna()`
- `.value_counts()`
- `.sort_index()`
- `.plot()`
- `.boxplot()`
- `.hist()`
- `.replace()`
- `.corr()`
- `.concat()`
- `.get_dummies()`

---

> Example of **Function Chaining** steps & explanation  

```python

# one line version
data_students[data_students.Gender==0].["Target"].value_counts(normalize=True).sort_index().plot(kind="bar", rot=0, title="Gender: Female", ylim=(0,.7))

# more readable version
data_students[data_students.Gender==0]\                           # filter data by gender column, with selected gender is 0
  ["Target"]\                                                     # Slicing the dataframe, retrieving column "Target"
  .value_counts(normalize=True)\                                  # Counting the categories in the "Target" column
  .sort_index()\                                                  # Sorting by categories in the "Target" column
  .plot(kind="bar", rot=0, title="Gender: Female", ylim=(0,.7))   # creating plot
  
```

---

# Library Set-up

In [ ]:
import pandas as pd
import numpy as np
from urllib.request import urlopen
import os, requests, json
from pprint import pprint
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

try:
  from dotenv import load_dotenv
  load_dotenv()
except:
  ! pip install -q python-dotenv
  from dotenv import load_dotenv
  load_dotenv()

try:
  from ucimlrepo import fetch_ucirepo
except:
  ! pip install -q ucimlrepo
  from ucimlrepo import fetch_ucirepo

try:
  from tabula import read_pdf
except:
  ! pip install -q tabula-py
  from tabula import read_pdf

# Gathering Data & Data Wrangling

## Open-Source

### UCI

In [ ]:
# fetch dataset
data_uci_id = 1
data_uci = fetch_ucirepo(id=data_uci_id)
data_uci

In [ ]:
type(data_uci)
dir(data_uci)
data_uci.keys()
data_uci["metadata"]

### GitHub

In [ ]:
# import url
GITHUB_URL = os.getenv("GITHUB_URL")

# read data with pandas
data_github = pd.read_csv(GITHUB_URL)

# show data
data_github.sample(10)

### Kaggle

In [ ]:
# Create a hidden directory called ".kaggle" in the user's home directory to store Kaggle API credentials
! mkdir ~/.kaggle
# ! mkdir C:\Users\ACER\.kaggle

# Copy the Kaggle API token file ("kaggle.json") to the newly created ".kaggle" directory
! cp kaggle.json ~/.kaggle/
# ! copy kaggle.json C:\Users\ACER\.kaggle\kaggle.json

# Set the permissions of the "kaggle.json" file to 600, making it readable and writable only by the user
! chmod 600 ~/.kaggle/kaggle.json
# ! icacls C:\Users\ACER\.kaggle\kaggle.json /grant Administrators:(OI)(CI)F

# Download the "credit-card-customers" dataset from Kaggle using the command line tool
! kaggle datasets download -d sakshigoyal7/credit-card-customers

# Unzip the downloaded dataset file ("credit-card-customers.zip") and extract its contents into a new directory called "data"
! unzip credit-card-customers.zip -d data

In [ ]:
data_kaggle = pd.read_csv("credit-card-customers.zip")
data_kaggle

## File / Others

### PDF

In [ ]:
PDF_NAME = os.getenv("PDF_NAME")
data_pdf = read_pdf(PDF_NAME, pages="all", encoding="ISO-8859-1")
data_pdf

### HTML

In [ ]:
URL = os.getenv("URL_2")
data_html = pd.read_html(URL)
data_html

## Scraping

### Case 1

In [ ]:
API_1 = os.getenv("API_1")

# get API response
response_api_1 = urlopen(API_1)
response_api_1 = json.loads(response_api_1.read())
response_api_1

In [ ]:
# convert API response to dataframe
data_api_1 = pd.DataFrame().from_records(response_api_1["series"])
data_api_1["data"] = data_api_1["data"].apply(lambda d: list(d)[0])
data_api_1

In [ ]:
# better plot
data_api_1\
  .set_index("name")\
  .sort_values("data")\
  .plot(
    kind="barh",
    title="Distribusi Prodi",
    ylabel="",
    legend=False
  )

#### Mini Hands-On Quiz  

> try implementing it with other available APIs! (_2 Minutes_)

In [ ]:
# YOUR ANSWER


### Case 2

In [ ]:
# try read html
data_api_2 = pd.read_html("https://www.banpt.or.id/direktori/prodi/pencarian_prodi.php")
data_api_2

In [ ]:
# import API
API_2 = os.getenv("API_2")

# get API response
response_api_2 = urlopen(API_2)
response_api_2 = json.loads(response_api_2.read())
response_api_2

In [ ]:
# convert API response to dataframe
data_api_2 = pd.DataFrame().from_records(response_api_2["data"])
data_api_2

In [ ]:
data_api_2[
    data_api_2[1].str.lower().str.contains("data")
  ]\
  [[1, 2]]\
  .value_counts()

#### Mini Hands-On Quiz  

> Try filtering to display/generate data with study programs(columns  `1`) containing the word "**data**" for **S1** level only! (_2 Minutes_)

In [ ]:
# YOUR ANSWER


### Case 3

In [ ]:
# import API
API_3A = os.getenv("API_3A")
API_3B = os.getenv("API_3B")

# reading data & response
data_api_3a = pd.read_json(API_3A)
display(data_api_3a)

response_api_3b = urlopen(API_3B)
response_api_3b = json.loads(response_api_3b.read())
display(response_api_3b)
data_api_3b = pd.DataFrame(response_api_3b["table"]).T
display(data_api_3b)

#### Mini Hands-On Quiz  

> does the `kode` column in the **data_api_3a** data match the **data_api_3b** data index? (_3 Minutes_)

In [ ]:
# YOUR ANSWER


In [ ]:
# renaming index
readable_index = data_api_3a.set_index("kode")["nama"].to_dict()
readable_index = {str(k):v for k,v in readable_index.items()}
data_api_3b.rename(readable_index, axis=0, inplace=True)
data_api_3b.drop(["psu", "status_progress"], axis=1, inplace=True)
data_api_3b

In [ ]:
data_api_3b.loc["JAWA TIMUR"].iloc[:-1].plot(kind="pie")

### Case 4

In [ ]:
sample_source = os.getenv("MATH_ITS_SAMPLE")
data_sample = pd.read_json(sample_source)
data_sample.sample(5)

In [ ]:
# more complex
from get_data_math import get_data_math

# getting data / scrap
data_math = get_data_math()

# inspect data
print("\n\n")
data_math.info()
data_math.sample(5)

**Remember This?**  

![](https://raw.githubusercontent.com/m-nanda/ilt-1/main/img/remember_this.png)

In [ ]:
# checking
column_to_check = ["divisions", "thesis_name", "thesis_type", "type"]

for col in column_to_check:
    display(data_math[col].value_counts())


In [ ]:
used_cols = ["ts", "title", "abstract", "keywords", "Year_", "Univ"]

data_math_filter = data_math[
  (data_math.thesis_name != "Master")
  &
  (data_math.type == "thesis")
]

data_math_filter = data_math_filter[used_cols]
data_math_filter.sort_values("Year_", inplace=True)
data_math_filter.reset_index(inplace=True, drop=True)
data_math_filter.info()
data_math_filter.sample(5)

[See Final Deliverable (Dashboard)](https://dashboard-mat.streamlit.app/)

---

# Data Analysis  

## Study Case: Data Students

In [ ]:
# import url
STUDENT_DATA_URL = os.getenv("STUDENT_DATA_URL")

# read data
data_students = pd.read_csv(STUDENT_DATA_URL, sep=";")

# show data samples
data_students.sample(10)

### Question / Hypotesis

- Is data clean?
- What else the status of the students in the dataset? (Univariate)
- what is the distribution of students' grades? (Univariate)
- Does Gender may affect graduation? (Bivariate)
- Are there certain features that have a strong correlation with student graduation? (Multivariate)

In [ ]:
# check data info
# YOUR CODE HERE

#### Data Inspection

In [ ]:
# check duplicate rows
# YOUR CODE HERE

In [ ]:
# check missing values if any
# YOUR CODE HERE

#### Univariate Analysis

> What else the status of the students in the dataset?

In [ ]:
# with unique / value counts
# YOUR CODE HERE

> what is the distribution of students' grades?

In [ ]:
# choose column to plot
column_to_plot = data_students.columns[-6]

# figure size settings
plt.figure(figsize=(12, 5))

# create simple boxplot
plt.subplot(121)
data_students[[column_to_plot]].boxplot()
plt.title(f"Boxplot")
plt.ylabel("Grade")
plt.xlabel("")

# create simple histogram
plt.subplot(122)
data_students[column_to_plot].hist(bins=25)
plt.title(f"Histogram")
plt.ylabel("Count")
plt.xlabel("Grade")

# show statistics
data_students[column_to_plot].describe()

#### Bivariate

> Does Gender may affect graduation? (Bivariate)

In [ ]:
# figure settings
plt.style.use("seaborn-darkgrid")
plt.figure(figsize=(8,4))

# Gender = 0
plt.subplot(121)
data_students[data_students.Gender==0]\
  ["Target"]\
  .value_counts(normalize=True)\
  .sort_index()\
  .plot(kind="bar", rot=0, title="Gender: Female", ylim=(0,.7))
plt.ylabel("Ratio")
plt.xlabel("Status")

# Gender = 1
plt.subplot(122)
data_students[data_students.Gender==1]\
  ["Target"]\
  .value_counts(normalize=True)\
  .sort_index()\
  .plot(kind="bar", rot=0, title="Gender: Male", ylim=(0,.7))
plt.ylabel("Ratio")
plt.xlabel("Status")
plt.show()

#### Multivariate

> Are there certain features that have a strong correlation with student graduation?

In [ ]:
# 1. basic

# make a copy of the dataframe
data_students_tmp = data_students.copy()

# create dict to convert Target to ordinal data
target_ordinal = {
  "Dropout":-1,
  "Enrolled":0,
  "Graduate":1
}

# convert string to ordinal
data_students_tmp["Target"] = data_students_tmp["Target"].replace(target_ordinal)

# create correlation
# YOUR CODE HERE

In [ ]:
# 2. more advance

# selecting only Target vs The rest column with sorting
data_students_tmp\
  .corr()\
  .loc["Target"]\
  .sort_values(ascending=False)\
  .iloc[1:]\
  .to_frame()

In [ ]:
# 3. advance

# create one hot encoding column Target
one_hot_encoding_target = pd.get_dummies(data_students["Target"], prefix="Target")
one_hot_encoding_target
pd.concat([data_students["Target"], one_hot_encoding_target], axis=1)

# concate one hot encoded Target to main dataframe and drop Target
data_students_tmp_2 = pd.concat([data_students.copy(), one_hot_encoding_target], axis=1)
data_students_tmp_2.drop("Target", axis=1, inplace=True)
data_students_tmp_2

# show correlation with formatting
data_students_tmp_2.corr()\
  .iloc[:-3,[-3,-2,-1]]\
  .sort_values("Target_Dropout", ascending=False)\
  .style\
  .background_gradient()

#### Mini Hands-On Quiz  

> Do the outlier 1st and 2nd semester grades need to be removed? Please include evidence and reasons! (_3 Minutes_)

In [ ]:
# YOUR ANSWER


---